# 🏥 고양이 의료 문서 분석 (Tesseract + LayoutParser)

이 노트북은 `data/은내과.pdf` 파일을 Tesseract + LayoutParser로 분석하여 의료 데이터를 추출합니다.

## 주요 기능
- PDF → 이미지 변환
- LayoutParser를 통한 문서 레이아웃 분석
- Tesseract OCR을 통한 테이블 데이터 추출
- 의료 검사 결과 구조화
- JSON 형태로 데이터 저장

## PaddleOCR과의 차이점
- **LayoutParser**: 문서의 레이아웃(테이블, 텍스트 블록 등)을 먼저 분석
- **Tesseract**: 구조화된 영역별로 정확한 OCR 수행
- **더 정확한 테이블 인식**: 의료 문서의 표 구조를 더 잘 파악

## 1. 환경 설정 및 라이브러리 로드

In [ ]:
# 시스템 패키지 업데이트
# sudo apt-get update

# Tesseract OCR 및 한국어 언어팩 설치
# sudo apt-get install tesseract-ocr tesseract-ocr-eng tesseract-ocr-kor

# 설치 확인
# tesseract --version
# tesseract --list-langs

In [3]:
# 기본 라이브러리 설치
!pip install PyMuPDF Pillow openai python-dotenv pytesseract pandas opencv-python

# LayoutParser 설치 (Detectron2 없이)
!pip install layoutparser scipy

# Detectron2는 별도로 설치 (옵션)
# !pip install torch torchvision
# !pip install 'git+https://github.com/facebookresearch/detectron2.git'

print("🔧 기본 라이브러리 설치 완료!")
print("⚠️ Tesseract는 시스템 레벨에서 별도 설치가 필요합니다.")
print("💡 터미널에서 다음 명령어를 실행하세요:")
print("   sudo apt-get update")
print("   sudo apt-get install tesseract-ocr tesseract-ocr-eng tesseract-ocr-kor")

  Using cached scipy-1.15.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached scipy-1.15.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.7 MB)
🔧 기본 라이브러리 설치 완료!
⚠️ Tesseract는 시스템 레벨에서 별도 설치가 필요합니다.
💡 터미널에서 다음 명령어를 실행하세요:
   sudo apt-get update
   sudo apt-get install tesseract-ocr tesseract-ocr-eng tesseract-ocr-kor


In [4]:
import os
import base64
import json
from typing import List, Dict, Any, Tuple
import fitz  # PyMuPDF
from PIL import Image, ImageEnhance, ImageFilter
import io
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd
from IPython.display import display
import re
from datetime import datetime
import numpy as np
import cv2

# Tesseract 및 LayoutParser
import pytesseract
import layoutparser as lp
from layoutparser.models import Detectron2LayoutModel
from layoutparser.elements import TextBlock, Rectangle

print("📦 라이브러리 로드 완료!")
print(f"🔍 Tesseract 버전: {pytesseract.get_tesseract_version()}")
print(f"📊 LayoutParser 버전: {lp.__version__}")

📦 라이브러리 로드 완료!
🔍 Tesseract 버전: 4.1.1
📊 LayoutParser 버전: 0.2.0


## 2. PDF 처리 및 이미지 전처리 함수 정의

In [5]:
def pdf_to_images(pdf_path: str, resolution_scale: float = 2.0) -> List[np.ndarray]:
    """PDF를 고품질 이미지로 변환 (OCR 최적화)"""
    doc = fitz.open(pdf_path)
    images = []
    
    print(f"📄 PDF 페이지 수: {len(doc)}")
    print(f"⚙️ 해상도 스케일: {resolution_scale}x")
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        
        # 고품질 변환을 위한 매트릭스 설정
        mat = fitz.Matrix(resolution_scale, resolution_scale)
        
        # 고품질 픽스맵 생성
        pix = page.get_pixmap(
            matrix=mat,
            alpha=False,
            annots=True,
            clip=None
        )
        
        # NumPy 배열로 변환 (LayoutParser 호환)
        img_data = pix.tobytes("png")
        img = Image.open(io.BytesIO(img_data))
        img_array = np.array(img)
        
        # BGR로 변환 (OpenCV 호환)
        if len(img_array.shape) == 3 and img_array.shape[2] == 3:
            img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        
        images.append(img_array)
        
        # 이미지 정보 출력
        size_mb = len(img_data) / (1024 * 1024)
        print(f"  📑 페이지 {page_num + 1}:")
        print(f"     📐 픽셀 크기: {img_array.shape[1]} x {img_array.shape[0]}")
        print(f"     💾 파일 크기: {len(img_data):,} bytes ({size_mb:.1f}MB)")
        print(f"     🎨 채널: {img_array.shape[2] if len(img_array.shape) == 3 else 1}")
        
        # 메모리 정리
        pix = None
    
    doc.close()
    return images

def preprocess_image_for_ocr(image: np.ndarray) -> np.ndarray:
    """OCR을 위한 이미지 전처리"""
    
    # 그레이스케일 변환
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image.copy()
    
    # 노이즈 제거
    denoised = cv2.medianBlur(gray, 3)
    
    # 대비 향상
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(denoised)
    
    # 이진화 (적응형 임계값)
    binary = cv2.adaptiveThreshold(
        enhanced, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    )
    
    # 형태학적 연산으로 텍스트 정리
    kernel = np.ones((1,1), np.uint8)
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    
    return cleaned

def save_image_preview(image: np.ndarray, filename: str):
    """이미지 미리보기 저장"""
    # BGR을 RGB로 변환 후 PIL Image로 변환
    if len(image.shape) == 3 and image.shape[2] == 3:
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(rgb_image)
    else:
        pil_image = Image.fromarray(image)
    
    # 미리보기용 크기 조정
    pil_image.thumbnail((800, 800))
    pil_image.save(filename)
    print(f"🖼️ 미리보기 저장: {filename}")

print("🔧 PDF 처리 및 이미지 전처리 함수 정의 완료!")

🔧 PDF 처리 및 이미지 전처리 함수 정의 완료!


## 3. LayoutParser 모델 초기화

In [6]:
def initialize_layout_model():
    """LayoutParser 모델 초기화"""
    
    print("🤖 LayoutParser 모델 로딩 중...")
    
    try:
        # PubLayNet 모델 사용 (논문/문서 레이아웃 분석에 최적화)
        model = lp.Detectron2LayoutModel(
            config_path='lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
            model_path='lp://PubLayNet/faster_rcnn_R_50_FPN_3x/model_final.pth',
            extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],  # 신뢰도 임계값
            label_map={0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"}
        )
        
        print("✅ LayoutParser 모델 로딩 완료!")
        print("📊 지원 레이아웃 요소: Text, Title, List, Table, Figure")
        
        return model
        
    except Exception as e:
        print(f"❌ LayoutParser 모델 로딩 실패: {e}")
        print("💡 대안: TableBank 모델 시도...")
        
        try:
            # TableBank 모델로 대체 시도 (테이블 특화)
            model = lp.Detectron2LayoutModel(
                config_path='lp://TableBank/faster_rcnn_R_50_FPN_3x/config',
                model_path='lp://TableBank/faster_rcnn_R_50_FPN_3x/model_final.pth',
                extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
                label_map={0: "Table"}
            )
            print("✅ TableBank 모델 로딩 완료!")
            return model
            
        except Exception as e2:
            print(f"❌ TableBank 모델도 실패: {e2}")
            print("⚠️ LayoutParser 없이 Tesseract만 사용합니다.")
            return None

# 모델 초기화
layout_model = initialize_layout_model()

🤖 LayoutParser 모델 로딩 중...
❌ LayoutParser 모델 로딩 실패: No module named 'detectron2'
💡 대안: TableBank 모델 시도...
❌ TableBank 모델도 실패: No module named 'detectron2'
⚠️ LayoutParser 없이 Tesseract만 사용합니다.


## 4. PDF 이미지 변환 및 미리보기

In [1]:
print("📂 파일 선택 (PDF 또는 이미지)")
print("=" * 40)

# 사용자 입력 받기 (data/ 폴더 기본 경로)
filename = input("📄 data/ 폴더 내 파일명을 입력하세요 (예: 은내과.PDF, 검사결과.png): ").strip()

# 빈 입력 시 기본값 사용
if not filename:
    filename = "은내과.PDF"
    print(f"🔹 기본 파일 사용: {filename}")

# data/ 경로와 결합
file_path = f"data/{filename}"
print(f"📁 파일 경로: {file_path}")

# 파일 확장자 확인
file_ext = filename.lower().split('.')[-1]
is_pdf = file_ext == 'pdf'
is_image = file_ext in ['png', 'jpg', 'jpeg', 'tiff', 'bmp']

print(f"📋 파일 형식: {file_ext.upper()}")

# output 폴더 생성
os.makedirs("output", exist_ok=True)

if is_pdf:
    # PDF 처리
    print("🔄 PDF → 이미지 변환 시작...")
    pdf_images = pdf_to_images(file_path, resolution_scale=2.0)
    
    # 미리보기 이미지 저장
    for i, img_array in enumerate(pdf_images):
        save_image_preview(img_array, f"output/page_{i+1}_preview.png")
    
    print(f"✅ PDF 변환 완료! {len(pdf_images)}개 페이지 처리됨")

elif is_image:
    # 이미지 파일 처리
    print("🖼️ 이미지 파일 로드 중...")
    
    # OpenCV로 이미지 로드
    img_array = cv2.imread(file_path)
    
    if img_array is None:
        # PIL로 재시도
        pil_img = Image.open(file_path)
        img_array = np.array(pil_img)
        if len(img_array.shape) == 3:
            img_array = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
    
    # 단일 이미지를 리스트에 담기
    pdf_images = [img_array]
    
    # 미리보기 이미지 저장
    save_image_preview(img_array, "output/image_preview.png")
    
    print(f"✅ 이미지 로드 완료! 1개 이미지 처리됨")

else:
    print(f"❌ 지원하지 않는 파일 형식입니다: {file_ext}")
    print("📋 지원 형식: PDF, PNG, JPG, JPEG, TIFF, BMP")
    pdf_images = []

if pdf_images:
    print(f"🎯 총 {len(pdf_images)}개 이미지가 분석 준비 완료!")
    print("📋 다음 섹션(LayoutParser + Tesseract 분석)을 실행할 수 있습니다!")
else:
    print("⚠️ 분석할 이미지가 없습니다. 올바른 파일을 선택해주세요.")

📂 파일 선택 (PDF 또는 이미지)
🔹 기본 파일 사용: 은내과.PDF
📁 파일 경로: data/은내과.PDF
📋 파일 형식: PDF


NameError: name 'os' is not defined

## 5. LayoutParser + Tesseract를 이용한 데이터 추출

In [14]:
def extract_medical_data_with_layoutparser_tesseract(image: np.ndarray, page_num: int = 0) -> pd.DataFrame:
    """LayoutParser + Tesseract로 혈액검사 데이터 추출 및 DataFrame 반환"""
    
    print(f"🔍 LayoutParser + Tesseract로 페이지 {page_num + 1} 분석 중...")
    
    try:
        # 이미지 전처리
        processed_image = preprocess_image_for_ocr(image)
        
        # 전처리된 이미지 저장 (디버깅용)
        cv2.imwrite(f"output/processed_page_{page_num + 1}.png", processed_image)
        print(f"💾 전처리 이미지 저장: output/processed_page_{page_num + 1}.png")
        
        all_texts = []
        layout_results = []
        
        # LayoutParser로 레이아웃 분석 (모델이 있는 경우)
        if layout_model is not None:
            print("📊 LayoutParser로 문서 레이아웃 분석 중...")
            
            # RGB 이미지로 변환 (LayoutParser 요구사항)
            rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            # 레이아웃 감지
            layout = layout_model.detect(rgb_image)
            
            print(f"📋 감지된 레이아웃 요소: {len(layout)}개")
            
            # 각 레이아웃 요소별로 OCR 수행
            for idx, block in enumerate(layout):
                block_type = block.type
                coords = block.coordinates
                
                print(f"  📦 요소 {idx+1}: {block_type} at {coords}")
                
                # 영역 추출
                x1, y1, x2, y2 = map(int, [coords[0], coords[1], coords[2], coords[3]])
                
                # 경계 체크
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(processed_image.shape[1], x2)
                y2 = min(processed_image.shape[0], y2)
                
                if x2 > x1 and y2 > y1:
                    roi = processed_image[y1:y2, x1:x2]
                    
                    # Tesseract OCR 수행
                    if block_type == "Table":
                        # 테이블의 경우 더 세밀한 설정
                        custom_config = r'--oem 3 --psm 6 -c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz가-힣.,/%-:()[]'
                    else:
                        # 일반 텍스트 설정
                        custom_config = r'--oem 3 --psm 6'
                    
                    try:
                        # 한국어 + 영어 OCR
                        text = pytesseract.image_to_string(
                            roi, 
                            lang='kor+eng',
                            config=custom_config
                        ).strip()
                        
                        if text:
                            all_texts.extend(text.split('\n'))
                            layout_results.append({
                                'type': block_type,
                                'bbox': (x1, y1, x2, y2),
                                'text': text
                            })
                            print(f"    📝 추출된 텍스트 ({len(text)} chars): {text[:50]}...")
                        
                    except Exception as ocr_error:
                        print(f"    ❌ OCR 오류: {ocr_error}")
        
        else:
            # LayoutParser 없이 Tesseract만 사용
            print("🔤 Tesseract만으로 전체 이미지 OCR 수행...")
            
            custom_config = r'--oem 3 --psm 6'
            
            try:
                # 한국어 + 영어 OCR
                full_text = pytesseract.image_to_string(
                    processed_image, 
                    lang='kor+eng',
                    config=custom_config
                ).strip()
                
                if full_text:
                    all_texts = full_text.split('\n')
                    print(f"📝 추출된 전체 텍스트 라인 수: {len(all_texts)}")
                
            except Exception as ocr_error:
                print(f"❌ Tesseract OCR 오류: {ocr_error}")
                all_texts = []
        
        # 빈 라인 제거 및 정리
        cleaned_texts = [text.strip() for text in all_texts if text.strip()]
        
        print(f"📝 정리된 텍스트 라인 수: {len(cleaned_texts)}")
        
        # 디버깅용: 추출된 텍스트 일부 출력
        print("🔍 추출된 텍스트 샘플:")
        for i, text in enumerate(cleaned_texts[:10]):  # 처음 10줄만
            print(f"  {i+1}: {text}")
        
        if len(cleaned_texts) > 10:
            print(f"  ... (총 {len(cleaned_texts)}줄 중 10줄만 표시)")
        
        # 검사일시 추출
        exam_date = extract_exam_date(cleaned_texts)
        print(f"📅 검사일시: {exam_date}")
        
        # 혈액검사 데이터 추출
        blood_tests = extract_blood_test_data(cleaned_texts)
        
        # DataFrame 생성
        if blood_tests:
            df = pd.DataFrame(blood_tests)
            df.attrs['exam_date'] = exam_date
            df.attrs['layout_results'] = layout_results
            
            print("✅ LayoutParser + Tesseract로 추출된 혈액검사 데이터:")
            display(df)
            print(f"\n📈 총 {len(df)}개 혈액검사 항목 추출됨")
            
            return df
        else:
            print("⚠️ 혈액검사 데이터를 찾을 수 없습니다.")
            empty_df = pd.DataFrame(columns=["혈액항목", "측정값", "단위"])
            empty_df.attrs['exam_date'] = exam_date
            empty_df.attrs['layout_results'] = layout_results
            return empty_df
            
    except Exception as e:
        print(f"❌ LayoutParser + Tesseract 분석 오류: {e}")
        import traceback
        print(f"🔍 상세 오류: {traceback.format_exc()}")
        empty_df = pd.DataFrame(columns=["혈액항목", "측정값", "단위"])
        empty_df.attrs['exam_date'] = "Not Found"
        empty_df.attrs['layout_results'] = []
        return empty_df

def extract_exam_date(texts: List[str]) -> str:
    """텍스트에서 검사일시 추출 (다양한 날짜 형식 지원)"""
    
    # 날짜 패턴 정의 (우선순위 순)
    date_patterns = [
        r'\b(20\d{2}[-:/]\d{1,2}[-:/]\d{1,2})\b',  # yyyy-mm-dd, yyyy/mm/dd, yyyy:mm:dd
        r'\b(\d{2}[-:/]\d{1,2}[-:/]\d{1,2})\b',    # yy-mm-dd, yy/mm/dd, yy:mm:dd
        r'\b(20\d{2}\.\d{1,2}\.\d{1,2})\b',       # yyyy.mm.dd
        r'\b(\d{2}\.\d{1,2}\.\d{1,2})\b',         # yy.mm.dd
        r'\b(20\d{2}\s+\d{1,2}\s+\d{1,2})\b',     # yyyy mm dd (공백)
        r'\b(\d{2}\s+\d{1,2}\s+\d{1,2})\b'        # yy mm dd (공백)
    ]
    
    found_dates = []
    
    for text in texts:
        # 생년월일 관련 키워드가 포함된 텍스트는 제외
        birth_keywords = ['생일', '생년월일', 'Birth', 'DOB', 'Born', '출생']
        if any(keyword in text for keyword in birth_keywords):
            continue
            
        # 검사일시 관련 키워드 우선 검색
        exam_keywords = ['검사일시', '검사날짜', 'Date', 'Time', '일시']
        is_exam_date_line = any(keyword in text for keyword in exam_keywords)
        
        for pattern in date_patterns:
            matches = re.findall(pattern, text)
            for match in matches:
                try:
                    # 날짜 형식 정규화
                    normalized_date = normalize_date(match)
                    if normalized_date:
                        # 검사일시 라벨이 있는 경우 우선순위 부여
                        priority = 1 if is_exam_date_line else 2
                        found_dates.append((normalized_date, priority, match))
                except:
                    continue
    
    if not found_dates:
        return "Not Found"
    
    # 우선순위로 정렬 (검사일시 라벨이 있는 것 우선, 그 다음 최신 날짜)
    found_dates.sort(key=lambda x: (x[1], -datetime.strptime(x[0], '%Y-%m-%d').timestamp()))
    
    return found_dates[0][0]

def normalize_date(date_str: str) -> str:
    """다양한 날짜 형식을 yyyy-mm-dd로 정규화"""
    
    # 구분자 통일 (-, /, :, ., 공백을 모두 -로)
    normalized = re.sub(r'[-:/.*+\s]+', '-', date_str.strip())
    
    try:
        parts = normalized.split('-')
        if len(parts) != 3:
            return None
            
        year, month, day = parts
        
        # 2자리 연도를 4자리로 변환
        if len(year) == 2:
            year_int = int(year)
            if year_int <= 30:  # 00-30은 2000년대
                year = f"20{year}"
            else:  # 31-99는 1900년대
                year = f"19{year}"
        
        # 월, 일을 2자리로 패딩
        month = month.zfill(2)
        day = day.zfill(2)
        
        # 유효한 날짜인지 검증
        datetime.strptime(f"{year}-{month}-{day}", '%Y-%m-%d')
        
        return f"{year}-{month}-{day}"
        
    except (ValueError, IndexError):
        return None

def extract_blood_test_data(texts: List[str]) -> List[Dict[str, str]]:
    """텍스트에서 혈액검사 데이터 추출"""
    
    # 혈액검사 항목 패턴 (대소문자 구분 없음)
    blood_test_patterns = {
        'RBC': r'\b(RBC|적혈구)\b',
        'WBC': r'\b(WBC|백혈구)\b',
        'HGB': r'\b(HGB|HB|헤모글로빈|혈색소)\b',
        'HCT': r'\b(HCT|헤마토크리트)\b',
        'PLT': r'\b(PLT|혈소판)\b',
        'MCV': r'\b(MCV)\b',
        'MCH': r'\b(MCH)\b',
        'MCHC': r'\b(MCHC)\b',
        'GLU': r'\b(GLU|포도당|혈당)\b',
        'CREA': r'\b(CREA|크레아티닌)\b',
        'BUN': r'\b(BUN|요소질소)\b',
        'ALT': r'\b(ALT|GPT)\b',
        'AST': r'\b(AST|GOT)\b',
        'ALP': r'\b(ALP|알카리포스파타제)\b',
        'TP': r'\b(TP|총단백)\b',
        'ALB': r'\b(ALB|알부민)\b'
    }
    
    # 단위 패턴
    unit_patterns = [
        r'\b(g/dL|mg/dL|×10³/μL|×10⁶/μL|/μL|fl|pg|%|U/L|IU/L)\b',
        r'\b(g/dl|mg/dl|x10³/μl|x10⁶/μl|/μl)\b',
        r'\b(mil/cmm|thou/cmm)\b'
    ]
    
    blood_tests = []
    
    # 각 텍스트 라인 분석
    for text in texts:
        text_upper = text.upper()
        
        # 혈액검사 항목 찾기
        for test_name, pattern in blood_test_patterns.items():
            if re.search(pattern, text_upper):
                # 숫자 값 추출 (소수점 포함)
                number_match = re.search(r'\b(\d+\.?\d*)\b', text)
                if number_match:
                    result = number_match.group(1)
                    
                    # 단위 추출
                    unit = ""
                    for unit_pattern in unit_patterns:
                        unit_match = re.search(unit_pattern, text)
                        if unit_match:
                            unit = unit_match.group(1)
                            break
                    
                    blood_tests.append({
                        "혈액항목": test_name,
                        "측정값": result,
                        "단위": unit
                    })
                    break
    
    # 중복 제거 (같은 항목이 여러 번 추출된 경우)
    seen_tests = set()
    unique_tests = []
    for test in blood_tests:
        test_key = test["혈액항목"]
        if test_key not in seen_tests:
            seen_tests.add(test_key)
            unique_tests.append(test)
    
    return unique_tests

print("🔧 LayoutParser + Tesseract 데이터 추출 함수 정의 완료!")

🔧 LayoutParser + Tesseract 데이터 추출 함수 정의 완료!


## 6. 의료 문서 분석 실행

In [15]:
print("🔍 LayoutParser + Tesseract 의료 문서 분석 시작...")
print("=" * 60)

all_results = []

for i, img_array in enumerate(pdf_images):
    print(f"\n📑 페이지 {i + 1} 분석 중...")
    print("-" * 40)
    
    # LayoutParser + Tesseract 분석
    analysis_result = extract_medical_data_with_layoutparser_tesseract(img_array, i)
    
    # 결과 저장
    if not analysis_result.empty:
        # JSON으로 저장
        result_data = {
            'page': i + 1,
            'exam_date': analysis_result.attrs.get('exam_date', 'Not Found'),
            'blood_tests': analysis_result.to_dict('records'),
            'layout_analysis': analysis_result.attrs.get('layout_results', [])
        }
        
        all_results.append(result_data)
        
        # 페이지별 결과 저장
        with open(f'output/page_{i+1}_layoutparser_tesseract_result.json', 'w', encoding='utf-8') as f:
            json.dump(result_data, f, ensure_ascii=False, indent=2)
        
        print(f"💾 결과 저장: output/page_{i+1}_layoutparser_tesseract_result.json")
    
    print(f"✅ 페이지 {i + 1} 분석 완료")

# 전체 결과 통합
if all_results:
    # 통합 JSON 저장
    final_result = {
        'total_pages': len(pdf_images),
        'processed_pages': len(all_results),
        'processing_method': 'LayoutParser + Tesseract',
        'timestamp': datetime.now().isoformat(),
        'results': all_results
    }
    
    with open('output/final_layoutparser_tesseract_analysis.json', 'w', encoding='utf-8') as f:
        json.dump(final_result, f, ensure_ascii=False, indent=2)
    
    print(f"\n🎯 전체 분석 완료!")
    print(f"📊 처리된 페이지: {len(all_results)}/{len(pdf_images)}")
    print(f"💾 최종 결과: output/final_layoutparser_tesseract_analysis.json")
    
    # 통계 출력
    total_blood_tests = sum(len(result['blood_tests']) for result in all_results)
    print(f"🧪 총 추출된 혈액검사 항목: {total_blood_tests}개")
    
    # 추출된 검사일시들
    exam_dates = [result['exam_date'] for result in all_results if result['exam_date'] != 'Not Found']
    if exam_dates:
        print(f"📅 검사일시: {', '.join(set(exam_dates))}")
    
else:
    print("⚠️ 분석 결과가 없습니다.")
    print("💡 다음을 확인해보세요:")
    print("   - 이미지 품질이 충분한가?")
    print("   - Tesseract가 올바르게 설치되었는가?")
    print("   - 한국어 언어팩이 설치되었는가?")


🔍 LayoutParser + Tesseract 의료 문서 분석 시작...

📑 페이지 1 분석 중...
----------------------------------------
🔍 LayoutParser + Tesseract로 페이지 1 분석 중...
💾 전처리 이미지 저장: output/processed_page_1.png
🔤 Tesseract만으로 전체 이미지 OCR 수행...
📝 추출된 전체 텍스트 라인 수: 34
📝 정리된 텍스트 라인 수: 27
🔍 추출된 텍스트 샘플:
  1: Date/Time: 2025-01-16 2 1:51:10                                         Performed By [ProCyte_DX]
  2: Name                          Unit            Min        Max      Result
  3: RBC                             10x6/uL            654          122         573 ww | 11] | 그
  4: 00                               %               303         523         261 ww [  J][ | 그
  5: HGB                              g/dl               98         162          78 ww [ [J [ | 그
  6: mcv                               개               35.9         531         ass normal [_ | OF | 그
  7: MCH                              09               118         173         136 normal [ | OD 1 고
  8: MCHC                            g/dl           

,혈액항목,측정값,단위
0,RBC,654,
1,HGB,98,g/dl
2,MCV,35.9,
3,MCH,09,
4,MCHC,28.1,g/dl
5,WBC,18.5,
6,PLT,151,



📈 총 7개 혈액검사 항목 추출됨
💾 결과 저장: output/page_1_layoutparser_tesseract_result.json
✅ 페이지 1 분석 완료

🎯 전체 분석 완료!
📊 처리된 페이지: 1/1
💾 최종 결과: output/final_layoutparser_tesseract_analysis.json
🧪 총 추출된 혈액검사 항목: 7개
📅 검사일시: 2025-01-16


## 7. 결과 비교 및 시각화

In [11]:
def visualize_layout_results(image: np.ndarray, layout_results: List[Dict], page_num: int):
    """LayoutParser 결과 시각화"""
    
    if not layout_results:
        print(f"📑 페이지 {page_num + 1}: 시각화할 레이아웃 결과가 없습니다.")
        return
    
    # 이미지 복사
    vis_image = image.copy()
    
    # 색상 정의 (BGR)
    colors = {
        'Text': (0, 255, 0),    # 녹색
        'Title': (255, 0, 0),   # 파랑
        'Table': (0, 0, 255),   # 빨강
        'List': (255, 255, 0),  # 청록
        'Figure': (255, 0, 255) # 자홍
    }
    
    # 각 레이아웃 요소에 박스 그리기
    for result in layout_results:
        bbox = result['bbox']
        element_type = result['type']
        
        x1, y1, x2, y2 = bbox
        color = colors.get(element_type, (128, 128, 128))
        
        # 박스 그리기
        cv2.rectangle(vis_image, (x1, y1), (x2, y2), color, 2)
        
        # 레이블 추가
        label = f"{element_type}"
        cv2.putText(vis_image, label, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    
    # 시각화 결과 저장
    output_path = f"output/page_{page_num + 1}_layout_visualization.png"
    cv2.imwrite(output_path, vis_image)
    
    print(f"🖼️ 레이아웃 시각화 저장: {output_path}")
    print(f"📊 감지된 요소: {', '.join([r['type'] for r in layout_results])}")

# 각 페이지의 레이아웃 결과 시각화
if all_results:
    print("🎨 레이아웃 분석 결과 시각화...")
    print("=" * 40)
    
    for i, (img_array, result) in enumerate(zip(pdf_images, all_results)):
        layout_results = result.get('layout_analysis', [])
        visualize_layout_results(img_array, layout_results, i)
    
    print("\n✅ 모든 시각화 완료!")
    print("📁 output/ 폴더에서 결과를 확인하세요.")
else:
    print("⚠️ 시각화할 결과가 없습니다.")

🎨 레이아웃 분석 결과 시각화...
📑 페이지 1: 시각화할 레이아웃 결과가 없습니다.

✅ 모든 시각화 완료!
📁 output/ 폴더에서 결과를 확인하세요.


## 8. 성능 비교 및 요약

### LayoutParser + Tesseract vs PaddleOCR 비교

| 특징 | LayoutParser + Tesseract | PaddleOCR |
|------|-------------------------|----------|
| **레이아웃 인식** | ✅ 우수 (구조화된 문서 분석) | ❌ 제한적 |
| **테이블 처리** | ✅ 매우 우수 | ⚠️ 보통 |
| **한국어 지원** | ✅ 우수 (Tesseract 한국어팩) | ✅ 우수 |
| **설치 복잡도** | ⚠️ 복잡 (여러 의존성) | ✅ 간단 |
| **처리 속도** | ⚠️ 느림 (2단계 처리) | ✅ 빠름 |
| **정확도** | ✅ 높음 (구조 기반) | ⚠️ 중간 |
| **의료문서 적합성** | ✅ 매우 적합 | ⚠️ 보통 적합 |

### 권장 사용 시나리오
- **LayoutParser + Tesseract**: 복잡한 의료문서, 정확한 테이블 추출이 필요한 경우
- **PaddleOCR**: 빠른 프로토타이핑, 간단한 텍스트 추출

In [12]:
print("📊 LayoutParser + Tesseract 분석 완료!")
print("=" * 50)
print("\n🎯 주요 성과:")
print(f"   📄 처리된 페이지: {len(all_results) if all_results else 0}개")
if all_results:
    total_tests = sum(len(r['blood_tests']) for r in all_results)
    print(f"   🧪 추출된 혈액검사: {total_tests}개")
    total_layouts = sum(len(r['layout_analysis']) for r in all_results)
    print(f"   📊 감지된 레이아웃 요소: {total_layouts}개")

print("\n📁 생성된 파일들:")
print("   📸 output/*_preview.png - 원본 이미지 미리보기")
print("   🔧 output/processed_*.png - 전처리된 이미지")
print("   🎨 output/*_layout_visualization.png - 레이아웃 분석 시각화")
print("   📄 output/page_*_layoutparser_tesseract_result.json - 페이지별 결과")
print("   📋 output/final_layoutparser_tesseract_analysis.json - 최종 통합 결과")

print("\n💡 다음 단계:")
print("   1. PaddleOCR 결과와 비교 분석")
print("   2. 더 정확한 후처리 규칙 개발")
print("   3. 의료진 검토를 통한 정확도 검증")

print("\n✨ 분석이 완료되었습니다!")

📊 LayoutParser + Tesseract 분석 완료!

🎯 주요 성과:
   📄 처리된 페이지: 1개
   🧪 추출된 혈액검사: 2개
   📊 감지된 레이아웃 요소: 0개

📁 생성된 파일들:
   📸 output/*_preview.png - 원본 이미지 미리보기
   🔧 output/processed_*.png - 전처리된 이미지
   🎨 output/*_layout_visualization.png - 레이아웃 분석 시각화
   📄 output/page_*_layoutparser_tesseract_result.json - 페이지별 결과
   📋 output/final_layoutparser_tesseract_analysis.json - 최종 통합 결과

💡 다음 단계:
   1. PaddleOCR 결과와 비교 분석
   2. 더 정확한 후처리 규칙 개발
   3. 의료진 검토를 통한 정확도 검증

✨ 분석이 완료되었습니다!
